In [1]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer
import re
from transformers import AdamWeightDecay
from datasets import Dataset
import math
import numpy as np
from transformers import DataCollatorForSeq2Seq
from transformers import TFAutoModelForSeq2SeqLM
import torch
import matplotlib.pyplot as plt
import evaluate

2023-11-22 07:59:38.512727: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 07:59:38.512754: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 07:59:38.514957: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 07:59:38.722339: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 07:59:39.603998: W tensorflow/compiler/

In [2]:
tf.config.experimental.list_physical_devices(device_type='GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
def json_to_dataset(filename):
    f = open(filename)
    df = pd.read_json(f)
    print(df.head())
    f.close()
    df['news'] = df['news'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
    #df = df.iloc[:10000]
    print(df.columns)
    print(df.head)
    dataset = Dataset.from_pandas(df)
    dataset = dataset.train_test_split(test_size=0.2)
    return dataset

In [10]:
train_data = json_to_dataset('Train_Headline_Generation.json')

                                                news  \
0  (Oct 7, 2014  12:40 PM CDT) As of Jan. 1, Walm...   
1  (Oct 29, 2013  8:15 AM CDT) Dax Shepard and Kr...   
2  (Mar 6, 2016  10:50 AM) Nancy Reagan, the help...   
3  (Aug 15, 2008  5:11 AM CDT) American Airlines ...   
4  (Apr 18, 2016  1:02 PM CDT) Ingrid Lyne, the S...   

                                            headline  
0   30K Walmart Part-Timers to Lose Health Insurance  
1     Dax Shepard: Wedding to Kristen Bell Cost $142  
2                            Nancy Reagan Dead at 94  
3  American Airlines Faces $7M Fine for Safety Vi...  
4   $222K Raised for Kids of Mom Dismembered on Date  
Index(['news', 'headline'], dtype='object')
<bound method NDFrame.head of                                                     news  \
0       As of Jan. 1, Walmart will no longer offer 30...   
1       Dax Shepard and Kristen Bell got married at t...   
2       Nancy Reagan, the helpmate, backstage adviser...   
3       American Ai

In [2]:
model_name ="czearing/article-title-generator"

In [5]:
prefix = "headline: "
#max_len = 3024
max_len = 2024
#max_l = 1028
max_l = 128

In [6]:
def preprocess_function(examples):   
    inputs = [prefix + doc for doc in examples["news"]]
    model_inputs = tokenizer(inputs, max_length=max_len, truncation=True)
    labels = tokenizer(text_target=examples["headline"], max_length=max_l, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

To apply the preprocess function over the entire dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
tokenized = train_data.map(preprocess_function, batched=True, #num_proc=4,
                        remove_columns=train_data['train'].column_names)

Map:   0%|          | 0/16925 [00:00<?, ? examples/s]

Map:   0%|          | 0/4232 [00:00<?, ? examples/s]

To create batches using DataCollatorForSeq2Seq, which dynamically pads the sentences to the longest length in a batch during collation. This is  more efficient than padding the whole dataset to the maximum length. 

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                           model=model_name,
                                           return_tensors="tf")
optimizer = AdamWeightDecay(learning_rate=5e-5, weight_decay_rate=0.01)

To load pretrained T5-base michau with AutoModelForSeq2SeqLM

In [13]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name, from_pt=True)

2023-11-21 17:11:39.057965: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:01:00.0, compute capability: 7.0
2023-11-21 17:11:39.058767: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31141 MB memory:  -> device: 1, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:25:00.0, compute capability: 7.0
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['lm_head.weight', 'encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

To convert datasets to the tf.data.Dataset format 

In [14]:
tf_train_set = model.prepare_tf_dataset(
    tokenized["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [15]:
model.compile(optimizer=optimizer)

In [16]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
history = model.fit(x=tf_train_set, validation_data=tf_test_set, callbacks=[callback],batch_size=6, epochs=10)
#model.fit(x=tf_train_set, validation_data=tf_test_set, batch_size=6, epochs=1)
#eval_loss = model.evaluate(tf_test_set)
#print(f"Finetuned Perplexity: {math.exp(eval_loss):.2f}")

Epoch 1/10
2820/2820 [==============================] - 701s 240ms/step - loss: 2.0786 - val_loss: 1.7570
Epoch 2/10
2820/2820 [==============================] - 678s 240ms/step - loss: 1.7563 - val_loss: 1.6929
Epoch 3/10
2820/2820 [==============================] - 677s 240ms/step - loss: 1.5524 - val_loss: 1.6682
Epoch 4/10
2820/2820 [==============================] - 676s 240ms/step - loss: 1.3876 - val_loss: 1.6684


In [4]:
f = open('Dev_Headline_Generation.json')
df = pd.read_json(f)
df['news'] = df['news'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
#df = df.iloc[45:50]
#df = df.iloc[100:500]
f.close()
test_data = Dataset.from_pandas(df)

In [5]:
def t5_predictions(data, tokenizer, model):
    text = prefix + data['news']
    inputs = tokenizer.encode_plus(text, return_tensors="pt").input_ids
    #input_ids = inputs['input_ids'].to(device)
    #attention_masks = inputs['attention_mask']
    
    outputs = model.generate(inputs,
                              max_length = 64, num_beams = 5, early_stopping = True)
    predictions = tokenizer.decode(outputs[0])
    result = re.sub("\<.*?\>","", predictions)
    return result

To load the ROUGE metric which stands for Recall-Oriented Understudy for Gisting Evaluation to calculates the similarity between the predicted headlines and actual headlines. ROUGE-N measures the number of matching n-grams between the model-generated text and an actual headline.



In [19]:
rouge = evaluate.load("rouge")

In [20]:
rougescores = []
predictions = []
for i in test_data:
    prediction = t5_predictions(i, tokenizer, model)
    answer = i['headline']
    print("Answer: ", answer)
    print("Prediction: ", prediction)
    rougescore = rouge.compute(predictions=[prediction], references=[answer])
    rougescores.append(rougescore['rougeL'])
    predictions.append(prediction)
    print("")

2023-11-21 18:07:47.212909: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f01f1b1ef40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-21 18:07:47.212971: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2023-11-21 18:07:47.212978: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2023-11-21 18:07:47.216990: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-21 18:07:47.233855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-11-21 18:07:47.241323: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:543] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation o

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Answer:  Yahoo Lays Off 2K Workers
Prediction:   Yahoo Cuts 14% of Jobs

Answer:  Auto Bosses to Press Pelosi for Another $25B
Prediction:   Detroit Big 3 to Ask Pelosi for $25B in Loans

Answer:  From Britain With Love: MI6 Seeks Minorities
Prediction:   MI6 Wants Women, Minorities in 007

Answer:  Deficit Breaks $1T Again
Prediction:   Obama Expects $1.29T Deficit for 2010

Answer:  3rd American With Ebola Lands in US
Prediction:   3rd US Aid Worker Sickened by Ebola Arrives in Nebraska

Answer:  Economy May Tank Palin's $30B Pipeline
Prediction:   Palin Pipeline Could Sink

Answer:  G20 Cops Caught on Tape Shoving Protester Who Died
Prediction:   G20 Protester Clubbed, Pulled to Ground

Answer:  US Soldier Held After Killing 5 at Baghdad Base
Prediction:   US Soldier Charged With Killing 5 at Stress Clinic

Answer:  Nintendo Chief Dies at 55
Prediction:   Nintendo President Dead at 55

Answer:  USS Pueblo Crew Awarded $65M
Prediction:   N. Korea Must Pay $65M to 4 US Crewmembers

An

Answer:  Arctic Hasn't Been This Warm in 44K Years
Prediction:   Arctic's Warming Trend Worst in 44K Years

Answer:  Paulson Wants Next $350B, But Bush May Not Ask for It
Prediction:   Paulson: Congress Need to Release Rest of $700B Bailout Fund

Answer:  Florida's Challenge: $1K to Killer of Longest Python
Prediction:   Florida Gives Snake Hunters $1K for Killing Longest

Answer:  Man Loses 14 Years of Work When Google Deletes His Blog
Prediction:   Google Removes 14-Year-Old Artist's Blog

Answer:  India Blames Pakistan in Killing of 5 Soldiers
Prediction:   India: Pakistan Killed 5 of Its Soldiers

Answer:  'Web 2.0' is English's Millionth Word
Prediction:   Web 2.0 Is the 1Mth Word in English

Answer:  Merkel Heads for 2nd Term, With Messy Coalition
Prediction:   Merkel in Pittsburgh With G20

Answer:  Rare Bug Kills SF Disease Researcher, 25
Prediction:   25-Year-Old Researcher Dies From Rare Bacteria

Answer:  Sonar Vehicle Seeking MH370 Crashes Into Volcano
Prediction:   MH370 S

Answer:  'Affluenza' Mom Off Hook for $3K Extradition Tab
Prediction:   Mom of Affluenza Teen Will Not Have to Pay $3K Back

Answer:  Father of Cryonics Robert Ettinger Dead at 92
Prediction:   Father of Cryonics Movement Dead at 92

Answer:  Fort Hood Rampage Lasted 8 Minutes
Prediction:   Fort Hood Gunman Killed 3 Soldiers, Wounded 16

Answer:  Merrill to Slash Bonuses 50%
Prediction:   Merrill to Cut Bonuses 50%

Answer:  Air Jordan Turns 23
Prediction:   Basketball's No. 23 Turns 23

Answer:  Iran Blockade Could Send Oil Up 50%
Prediction:   Oil Prices Could Jump 50% if Iran Closes Strait

Answer:  Insider Shooting Kills 2 US Troops
Prediction:   Insider Attack Kills 5 in Afghanistan

Answer:  One in 30 US Kids Is Homeless
Prediction:   1 Child in Every 30 US Kids Homeless

Answer:  2 Candidates, 2 Americas: Old Values vs. New
Prediction:   It's a Battle of the Old vs. the New

Answer:  Google, Facebook Eye Twitter, Price Tag: $10B
Prediction:   Facebook, Google Considering $10B Tw

...

Answer:  Rising Broadway Star Dies Suddenly at 13
Prediction:   Broadway Rising Star, 13, Dies of Asthma

Answer:  Over 700 Parents 'Ineligible' for Child Reunification
Prediction:   1.8K Kids Separated at Border Reunited

Answer:  First US Baby of 2018 Wasn't Born in Any of the 50 States
Prediction:   2018's 1st Baby Is Here

Answer:  Source: Robert Pattinson Has COVID-19
Prediction:   Pattinson Tested Positive for Covid-19 3 Days Ago

Answer:  Leon Redbone, Musician of Mystery, Dies at 69
Prediction:   Leon Redbone Dead at 69

Answer:  Dow Rises 400 on China Trade Talks
Prediction:   Dow Ends Day Up 400

Answer:  Red Sox Win 4th World Series Title in 15 Seasons
Prediction:   Red Sox Win 4th World Series

Answer:  8-Year-Old Takes Mom's Car, Hits 87mph on Highway
Prediction:   8-Year-Old Takes Mom's Car, Drives Joyride

Answer:  3rd NBA Player Tests Positive
Prediction:   3rd NBA Player Tests Positive for COVID-19

Answer:  Almost 50 Years Later, We Know Who Killed Her
Prediction:   D

Answer:  California Shakes Up 2020 Primary Contests
Prediction:   California Pushes Primary Up 3 Months Early

Answer:  5 Rwandan Rape Survivors Stood Up and Changed History
Prediction:   Survivors of 1990s Rape Help Establish Rape as a Crime

Answer:  Jury: Michael Jackson Estate Owes Quincy Jones $9.4M
Prediction:   Jackson Estate Owes Quincy Jones $9.4M

Answer:  Grandma, Cat Survive 5 Days Stuck in Mountains
Prediction:   85-Year-Old Gets Stuck 20 Miles Down Mountain Road

Answer:  Apple Breaks 15-Year Streak of Increasing Sales
Prediction:   Apple's 2016 Sales Drop $18B

Answer:  Alanis Victim of 2nd Multimillion-Dollar Theft
Prediction:   Burglars Steal $2M in Jewelry From Alanis Morissette

Answer:  Lonely Veteran, 89, Who Placed Ad Looking for Work Starts New Job
Prediction:   Army Veteran, 89, Starts New Job After Newspaper Ad

Answer:  US Spies Heard Russians Discussing Trump Aides in 2015
Prediction:   Russia Re-Examines 2015 Conversations With Trump Associates

Answer:  The

Answer:  Mastered 5 Fruits and Veggies a Day? Now Go for 10
Prediction:   10 Servings of Fruit and Vegetables a Day Could Save 7.8M

Answer:  Their Film Ran in 14 Theaters. Then Robert Pattinson Called
Prediction:   Robert Pattinson's 'Good Time' Is Just the First 5 Minutes

Answer:  17th-Century Painting Cut From Frame in Quake-Hit Church
Prediction:   Thieves Steal 17th-Century Painting From Damaged Church

Answer:  Remains of 5 Archbishops of Canterbury Found, by Mistake
Prediction:   5 Archbishops Interred in 30 Lead Coffins

Answer:  Arkansas Carries Out 1st US Double Execution Since 2000
Prediction:   Arkansas Carries Out First Double Execution Since 2000

Answer:  A New Year, 2 New Star Engagements
Prediction:   Conrad's Great-Granddaughter Engages in Colo.

Answer:  After 130 Years, a 'Historic Homecoming' in Banff
Prediction:   16 Bison Moved to Canada's Oldest National Park

Answer:  How One Man Made $2.5M Betting on the World Series
Prediction:   Vegas Dave Oancea: I Won $2.

Answer:  Fewer Than 20 da Vinci Paintings Survive. A New Fate for One
Prediction:   Leonardo da Vinci Could Fetch $100M

Answer:  14 Photos From the Moment Trump Became No. 45
Prediction:   Trump Is 45th President

Answer:  Suspect Who Let Partner Drown Nabbed 5 Years Later: Cops
Prediction:   Man's Body Found in Florida Canal 5 Years Ago

Answer:  35 Gang Members Arrested in Long Island Slayings
Prediction:   35 Suspected Members of NY Gang Arrested

Answer:  Couple Together for 6 Decades Carry Out Suicide Pact: Cops
Prediction:   88-Year-Old, 86-Year-Old Found Dead in NYC Apartment

Answer:  Secret Gov't Airline Needs Someone to Fly to Area 51
Prediction:   Secret Airlines Hiring for Flight Attendant in Las Vegas

Answer:  Woman 'Wins' $43M on Slots; Casino Says Not So Fast
Prediction:   Mom of 4 Loses $43M, Gets Steak Dinner

Answer:  5 States That Need to Take a Chill Pill
Prediction:   5 Most Stressed States

Answer:  11-Year-Old Saves Family From Bear Attack
Prediction:   Boy, 11

Answer:  School District to Pay $100K Over 'Drunk Goggles' Injury
Prediction:   School District to Pay $100K to Teen Injured in Drunk Experiment

Answer:  Thieves Steal $400K Worth of Stuff From Venus Williams' Home
Prediction:   Burglars Steal $400K From Venus Williams' Florida Home

Answer:  Paris Thieves Nab $5.6M in Highway Heist
Prediction:   Paris Robbers Steal $5.6M From Highway

Answer:  Sri Lanka Mudslides: 151 Dead, 102K Displaced
Prediction:   111 More Missing as Sri Lanka Death Toll Hits 151

Answer:  House GOP to Slash Corporate Tax Rate From 39% to 20%
Prediction:   GOP Cuts Income Tax Rate for Richest Earners

Answer:  Trump Welcomes 21K to White House Easter Egg Roll
Prediction:   Trump Opens 1st White House Easter Egg Roll

Answer:  Inmates Escapes Maximum Security for 2nd Time
Prediction:   Inmate Breaks Out of SC Prison for 2nd Time

Answer:  Report: Williams Got 0.07% of Male Co-Star's Pay for Reshoot
Prediction:   'All the Money in the World' Reshoot Cost Mark Wahl

In [22]:
df = pd.DataFrame()
df['predictions'] = predictions
df['rougescores'] = rougescores
df.head()

,predictions,rougescores
0,3 Dead in California Quarry Shooting Spree,0.500000
1,Dow Rises 305 on Election Day,0.333333
2,The Interview Is Now Available for $14.99,0.266667
3,"10-Year-Old Confessioned to Killing Woman, 90",0.125000
4,Tobacco Companies Didn't Warn About Radiation...,0.444444


In [23]:
df.to_excel('hpc_cv21.xlsx')

In [24]:
len(df.loc[df['rougescores'] >= 0.5])

700

In [25]:
len(df.loc[df['rougescores'] >= 0.75])

187

In [26]:
len(df.loc[df['rougescores'] >= 1])

36

In [27]:
df['rougescores'].describe()

count    2365.000000
mean        0.383088
std         0.225403
min         0.000000
25%         0.222222
50%         0.352941
75%         0.533333
max         1.000000
Name: rougescores, dtype: float64

In [10]:
from evaluate import load
bertscore = load("bertscore")

In [12]:
df = pd.read_excel('hpc_cv21.xlsx')
df['answers'] = test_data['headline']
df.head()

,Unnamed: 0,predictions,rougescores,answers
0,0,3 Dead in California Quarry Shooting Spree,0.500000,3rd Victim Dead in Quarry Shooting; Manhunt St...
1,1,Dow Rises 305 on Election Day,0.333333,Stocks Up 305 in Election Rally
2,2,The Interview Is Now Available for $14.99,0.266667,You Can Watch The Interview at 1pm
3,3,"10-Year-Old Confessioned to Killing Woman, 90",0.125000,"Murder Suspect, 10, Will Stay in Adult Jail"
4,4,Tobacco Companies Didn't Warn About Radiation...,0.444444,Tobacco Firms Knew of Radiation in Cigs—in 1959


In [13]:
df['bertscores'] = df.apply(lambda x: bertscore.compute(predictions=[x['predictions']], references=[x['answers']],
                                                       rescale_with_baseline=True, lang="en", batch_size=1), axis=1)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
df['bert_f1'] = [d.get('f1')[0] for d in df['bertscores']]
df.head()

,Unnamed: 0,predictions,rougescores,answers,bertscores,bert_f1
0,0,3 Dead in California Quarry Shooting Spree,0.500000,3rd Victim Dead in Quarry Shooting; Manhunt St...,"{'precision': [0.514030933380127], 'recall': [...",0.386882
1,1,Dow Rises 305 on Election Day,0.333333,Stocks Up 305 in Election Rally,"{'precision': [0.33300623297691345], 'recall':...",0.276598
2,2,The Interview Is Now Available for $14.99,0.266667,You Can Watch The Interview at 1pm,"{'precision': [0.17242328822612762], 'recall':...",0.126363
3,3,"10-Year-Old Confessioned to Killing Woman, 90",0.125000,"Murder Suspect, 10, Will Stay in Adult Jail","{'precision': [0.16259188950061798], 'recall':...",0.171261
4,4,Tobacco Companies Didn't Warn About Radiation...,0.444444,Tobacco Firms Knew of Radiation in Cigs—in 1959,"{'precision': [0.5463172197341919], 'recall': ...",0.458634


In [17]:
df.to_excel('sub2_cz.xlsx')